In [ ]:
import pandas as pd
import numpy as np
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.simplefilter('ignore')


This notebook is  'Ubiquant Market Prediction Competition' EDA for biginner's.

The goal of 'Ubiquant Market Prediction Competition' is for forecast the return rates of investment.

In this competition, big data over 18GB have to be treated.

So, I reccomend to read as parguet not csv, when reading datasets of this competition,because kaggle notebooks don't make crash.


このノートブックは、「UbiquantMarketPredictionCompetition」の初心者向けEDAです。
「Ubiquant Market Prediction Competition」の目標は、投資収益率を予測することです。

このコンテストでは18GBを超えるビッグデータを処理する必要があります。

したがって、このコンテストのデータセットを読むときは、kaggleノートブックがクラッシュしないため、parguetとして読むことをお勧めします。


reflection:[⏫ Fast Data Loading and Low Mem with Parquet Files](https://www.kaggle.com/robikscube/fast-data-loading-and-low-mem-with-parquet-files)

In [ ]:
train_df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

# About Train

* row_id : connect time_id and investment_id　(time_idおよびinvestment_id)

* time_id : time index(時間インデックス)

* investment_id : investment's id(投資のID)

* target : an investment's return rate(投資の収益率)

* f_0~f_299 : features(特徴量)



In [ ]:
train_df.describe()

In [ ]:
print('the number of investment_id: ',len(train_df['investment_id'].unique()))
print('the number of time_id: ',len(train_df['time_id'].unique()))
print('the number of features: ',len([i for i in train_df.columns if 'f_' in i]))

the number of investments are 3579, the number of features are 300, and time index is 1211.

投資の数は3579、機能の数は300、時間インデックスは0~1211です。

# No missing data 

Let's check the number of missing data.

In [ ]:
null_data=train_df.isnull().sum()
null_list=[]
for i in null_data.index:
    if null_data[i]!=0:
        null_list.append(null_data[i])
        
        
if len(null_list)==0:
    print('There are no null.')

There are no null.

# About Target

<p style="text-decoration:underline; text-decoration-color:#FF0000;">"In this competition, you’ll build a model that forecasts an investment's return rate."</p>

<p style="text-decoration:underline; text-decoration-color:#FF0000;">"このコンテストでは、投資の収益率を予測するモデルを構築します。"</p>

reference:[Ubiquant EDA and Baseline](https://www.kaggle.com/ilialar/ubiquant-eda-and-baseline)

In [ ]:
for number in range(100,3700,500):
    try:
        example=train_df[train_df['investment_id']==number]
        plt.plot(example['time_id'],example['target'])
        plt.title('investment_id='+str(number))
        plt.show()
        
    except:print(number,'is not found.')

Some time index are missing.

いくつかの時間インデックスが欠落しています。

In [ ]:
list_id=np.sort(train_df['investment_id'].unique())

In [ ]:
mean_list=[]
for i in range(300):
        number=list_id[i]
        example=train_df[train_df['investment_id']==number]
        example['target'].hist(bins = 100, alpha = 0.1, figsize = (20,10))
        mean_list.append(example['target'].mean())
mean_=np.array(mean_list).mean()
plt.axvline(mean_, ls = "--",color = "navy")
plt.title('target mean='+str(mean_))
plt.show()

In [ ]:
median_list=[]
for i in range(300):
        number=list_id[i]
        example=train_df[train_df['investment_id']==number]
        example['target'].hist(bins = 100, alpha = 0.1, figsize = (20,10))
        median_list.append(example['target'].median())
median_=pd.Series(median_list).median()
plt.axvline(median_, ls = "--", color = "red")
plt.title('target median='+str(median_))
plt.show()

In [ ]:
print('target mean=',train_df['target'].mean())
print('target median=',train_df['target'].median())

The median and mean of target are little negative
ターゲットの中央値と平均値は少しマイナスです。

Next let's check the target more. 
次に、ターゲットをさらに確認しましょう

In [ ]:
df_all=pd.DataFrame()
for number in list_id:
        example=train_df[train_df['investment_id']==number]
        example.index=example['time_id']
        example['inv_'+str(number)]=example['target']
        df_all=pd.concat([df_all,example['inv_'+str(number)]],axis=1)

In [ ]:
df_all.describe()

In [ ]:
plt.figure(figsize=(15,10))
plt.ylabel("length of time index")
plt.xlabel("ivestment_id")
plt.bar(df_all.count().index,df_all.count().values)

## Augmented Dickey-Fuller (ADF) test

Use the Augmented Dickey Fuller (ADF) test with some investment_id targets to see how constant the rate of change in your investment is.


いくつかのinvestment_idのターゲットで、拡張ディッキーフラー（ADF）テストを使用して投資の変化率の定常性を確認しましょう。

About Augmented Dickey Fuller (ADF) :[Time Series: Interpreting ACF and PACF](https://www.kaggle.com/iamleonie/time-series-interpreting-acf-and-pacf)

In [ ]:
import statsmodels as sm
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima_model import ARMA

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller

# good stationarity

def check_stationarity(series):
    # Copied from https://machinelearningmastery.com/time-series-data-stationary-python/

    result = adfuller(series.values)

   #print('ADF Statistic: %f' % result[0])
    #print('p-value: %f' % result[1])
    #print('Critical Values:')
    #for key, value in result[4].items():
        #print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

for number in range(100,3700,100):
    try:
        print('investment_id=',number)
        example=train_df[train_df['investment_id']==number]

        check_stationarity(example['target'])

        example['target_diff'] = example['target'].diff().fillna(0)

        check_stationarity(example['target_diff'])
        print('-----------------')
    except:print(number,'is not found.')

Target seems to have a lot of data that stationarity is confirmed.
Targetは定常性が確認されるデータが多いようです。

# About Features

So let's see the 300 features.

300の特徴量を見てみましょう。

In [ ]:
train_df.describe()

In [ ]:
rows, cols = 5, 10  
fig, aX_invs = plt.subplots(ncols=cols, nrows=rows, figsize=(18, 10)) 
for i in range(50):
    r = i // cols
    c = i % cols
    
    f = 'f_'+str(i)
    aX_invs[r, c].hist(train_df[f],bins = 100)
    aX_invs[r, c].set_title(f)
    aX_invs[r, c].get_xaxis().set_visible(False)
    aX_invs[r, c].get_yaxis().set_visible(False)
plt.show()

### statistical distribution(統計分布)

Mostly, statistical distributions　of unimodal.But sometimes they are included  bimodal and multimodal.

ほとんどの場合、ユニモーダルの統計分布ですが、バイモーダルとマルチモーダルが含まれる場合もあるようです。


In [ ]:
f_list = ['f_105','f_106']
for f in f_list:
    train_df[f].hist(bins = 100, figsize = (10,5),color='green')
    plt.title(f)
    plt.show()

* bimodal=having two modes, of a statistical distribution.(having two maxima)

* multimodal=having more than two modes.

* 二峰性 = 統計的分布の2つのモードを持っている(2つの最大値を持っている。)

* 多峰性 = 2つ以上のモードを持っている

In [ ]:
f_list = ['f_48','f_296']
for f in f_list:
    train_df[f].hist(bins = 100, figsize = (10,5),color='orange')
    plt.title(f)
    plt.show()

## Features vs target

Let's see every features once with a specific investment_id.

特定のinvestment_idを使用してすべての機能を一度見てみましょう。

In [ ]:
#You can change the investment_id.
data_200=train_df[train_df['investment_id']==200]

#all features
rows, cols = 60, 5
fig, aX_invs = plt.subplots(ncols=cols, nrows=rows, figsize=(18, 120)) 
for i in range(300):
    r = i // cols
    c = i % cols
    
    f = 'f_'+str(i)
    #plt.scatter(data_200.index,data_200[f])
    plt.title(f)
    aX_invs[r, c].scatter(data_200[f],data_200['target'],alpha=0.5)
    aX_invs[r, c].set_title(f)
    aX_invs[r, c].get_xaxis().set_visible(False)
    aX_invs[r, c].get_yaxis().set_visible(False)
plt.show()

We can see they don't have clear correlations with the target.


ターゲットとの明確な相関関係がないことがわかります。



If you look at it so far, I think it's similar to the Jane Street competition in the past competitions.

The JaneStreet competition was a classification, but there are many similarities such as the number of features and the method of distribution.

ここまで見てみると過去のコンペティションでJaneStreetのコンペティションに少し似ていると思います。
JaneStreetコンペティションはクラス分類でしたが、特徴量の多さや分散の仕方など似ている点が多いと感じます。